In [1]:
#Shree Ganesha

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import datetime
import os
from skimage.io import imread
from keras import backend as K

In [ ]:
from skimage.transform import resize as imresize
from PIL import Image, ImageFilter, ImageEnhance
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
##Ignoring warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
np.random.seed(30)

import random as rn
rn.seed(30)

import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
train_doc = np.random.permutation(open('/kaggle/input/upgrad-gesture-recognistion/Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/kaggle/input/upgrad-gesture-recognistion/Project_data/Project_data/val.csv').readlines())
#setting experimental batch size of 20
batch_size = 20
##Setting image dimension =120x120
xdim, ydim = 120, 120
img_idx_sel = range(0,25,2)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize

def generator(source_path, folder_list, batch_size, img_idx_sel, xdim, ydim):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = img_idx_sel  # creating a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y, z = ydim, xdim  # note the change in order of ydim and xdim
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size  # calculate the number of batches based on the total number of folders

        for batch in range(num_batches):  # iterate over the batches
            batch_data = np.zeros((batch_size, x, y, z, 3))  # initialize batch data
            batch_labels = np.zeros((batch_size, 5))  # initialize batch labels
            for folder in range(batch_size):  # iterate over the batch
                imgs = os.listdir(source_path + '/' + t[folder + (batch * batch_size)].split(';')[0])  # read all the images in the folder
                for idx, item in enumerate(img_idx):  # Iterate over the frames/images of a folder to read them in
                    image = imread(source_path + '/' + t[folder + (batch * batch_size)].strip().split(';')[0] + '/' + imgs[item]).astype(np.float32)

                    if image.shape[1] > image.shape[0]:
                        diff_px = image.shape[1] - image.shape[0]
                        crop_start = diff_px // 2
                        crop_end = crop_start + image.shape[0]
                        image = image[:, crop_start:crop_end]
                    elif image.shape[0] > image.shape[1]:
                        diff_px = image.shape[0] - image.shape[1]
                        crop_start = diff_px // 2
                        crop_end = crop_start + image.shape[1]
                        image = image[:, crop_start:crop_end]

                    resized_im = resize(image, (y, z))  # resize the image

                    # normalise and feed in the image
                    batch_data[folder, idx, :, :, 0] = resized_im[:, :, 0] / 255
                    batch_data[folder, idx, :, :, 1] = resized_im[:, :, 1] / 255
                    batch_data[folder, idx, :, :, 2] = resized_im[:, :, 2] / 255

                batch_labels[folder, int(t[folder + (batch * batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels  # yield the batch data and labels


In [ ]:
#!ls /kaggle/input/upgrad-gesture-recognistion/Project_data/Project_data/train

In [13]:
!ls -l /kaggle/working/model_init4_2024-04-02_16_15_17
!chmod 755 /kaggle/input/upgrad-gesture-recognistion/*
!cp /kaggle/working/model_init4_2024-04-02_16_15_17/model4-00027-0.44512-0.82206-0.49983-0.81000.keras /kaggle/input/upgrad-gesture-recognistion/


total 16172
-rw-r--r-- 1 root root 1273305 Apr  3 14:45 model4-00001-1.66745-0.39559-2.53384-0.23000.keras
-rw-r--r-- 1 root root 1273305 Apr  3 14:45 model4-00003-1.08986-0.53235-1.74805-0.20000.keras
-rw-r--r-- 1 root root 1273305 Apr  3 14:45 model4-00004-1.03704-0.57647-1.55253-0.33000.keras
-rw-r--r-- 1 root root 1273305 Apr  3 14:45 model4-00005-0.98937-0.60294-1.46682-0.39000.keras
-rw-r--r-- 1 root root 1273306 Apr  3 14:45 model4-00016-0.49422-0.79412-1.34974-0.47000.keras
-rw-r--r-- 1 root root 1273306 Apr  3 14:45 model4-00017-0.52767-0.77941-1.22395-0.57000.keras
-rw-r--r-- 1 root root 1273306 Apr  3 14:45 model4-00018-0.46948-0.80147-0.84139-0.69000.keras
-rw-r--r-- 1 root root 1273306 Apr  3 14:45 model4-00019-0.50223-0.78824-0.69380-0.74000.keras
-rw-r--r-- 1 root root 1273306 Apr  3 14:45 model4-00020-0.52646-0.77500-0.61870-0.77000.keras
-rw-r--r-- 1 root root 1273306 Apr  3 14:45 model4-00021-0.51985-0.78971-0.56018-0.77000.keras
-rw-r--r-- 1 root root 1273306 Apr  3 

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/kaggle/input/upgrad-gesture-recognistion/Project_data/Project_data/train'
val_path = '/kaggle/input/upgrad-gesture-recognistion/Project_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20    ### Setting initial 20 epochs to test
print ('# epochs =', num_epochs)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Bidirectional, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from tensorflow.keras.layers import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#Input Shape
input_shape = (len(img_idx_sel), xdim, ydim, 3)

In [ ]:
### Test Model #1

In [ ]:
model = Sequential()
model.add(Conv3D(64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=2))
model.add(Conv3D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [ ]:
print ("Model summary : Test #1 ",model.summary())

In [ ]:
optimiser = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
train_generator = generator(train_path, train_doc, batch_size,img_idx_sel, xdim, ydim)
val_generator = generator(val_path, val_doc, batch_size,img_idx_sel, xdim, ydim)

In [ ]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

# if not os.path.exists(model_name):
#     os.mkdir(model_name)

# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')

# LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1)   # write the REducelronplateau code here
# callbacks_list = [checkpoint, LR]

In [ ]:
#####
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

model_name = 'model_init_' + datetime.now().strftime('%Y-%m-%d_%H_%M_%S') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
optimiser = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])



In [ ]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                        callbacks=callbacks_list, validation_data=val_generator,
                        validation_steps=validation_steps, class_weight=None, initial_epoch=0)

In [ ]:
##Checking GPU details
gpu_memory_info = tf.config.experimental.get_memory_info("GPU:0")
gpu_memory_info

In [ ]:
""

In [ ]:
plt.figure(figsize=(20,6))
ax1 = plt.subplot(121)
ax1 = plt.plot(history.history['loss'])
ax1 = plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')
ax2 = plt.subplot(122)
ax2 = plt.plot(history.history['categorical_accuracy'])
ax2 = plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')

In [ ]:
history

In [ ]:
type(history)

In [ ]:
model2 = Sequential()

model2.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model2.add(Conv3D(64, kernel_size=3, activation='relu'))
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model2.add(Conv3D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2)))

model2.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2)))

model2.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model2.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2)))

model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.Adam(learning_rate=0.01)
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])



In [ ]:
train_generator = generator(train_path, train_doc, batch_size,img_idx_sel, xdim, ydim)
val_generator = generator(val_path, val_doc, batch_size,img_idx_sel, xdim, ydim)
##can be skipped if ran fully
#####
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_name = 'model_init2_' + datetime.now().strftime('%Y-%m-%d_%H_%M_%S') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model2-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]



In [ ]:
print ("Model summary : Test #2 ",model2.summary())

In [ ]:
import os
import tensorflow as tf


# Set TensorFlow to use GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU device 0

# Train the model with GPU
history2 = model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=40, verbose=1,
                       callbacks=callbacks_list, validation_data=val_generator,
                       validation_steps=validation_steps, class_weight=None,
                       initial_epoch=0, batch_size=batch_size)

In [ ]:
plt.figure(figsize=(20,6))

ax1 = plt.subplot(121)
ax1 = plt.plot(history2.history['loss'])
ax1 = plt.plot(history2.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')

ax2 = plt.subplot(122)
ax2 = plt.plot(history2.history['categorical_accuracy'])
ax2 = plt.plot(history2.history['val_categorical_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower left')

In [ ]:
### Model #3

In [ ]:
model3 = Sequential()

model3.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))

model3.add(Conv3D(64, kernel_size=3, activation='relu'))
model3.add(MaxPooling3D(pool_size=(2, 2, 2)))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

model3.add(Conv3D(128, kernel_size=3, activation='relu'))
model3.add(MaxPooling3D(pool_size=(1, 2, 2)))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

model3.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model3.add(MaxPooling3D(pool_size=(1, 2, 2)))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

model3.add(GlobalAveragePooling3D())
model3.add(Dense(512, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dense(5, activation='softmax'))

In [ ]:
print ("Model summary : Test #3 ",model3.summary())

In [ ]:
optimiser = optimizers.Adam(learning_rate=0.01)
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])



In [ ]:
train_generator = generator(train_path, train_doc, batch_size,img_idx_sel, xdim, ydim)
val_generator = generator(val_path, val_doc, batch_size,img_idx_sel, xdim, ydim)
##can be skipped if ran fully
#####
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_name = 'model_init3_' + datetime.now().strftime('%Y-%m-%d_%H_%M_%S') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model3-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]



In [ ]:
import os
import tensorflow as tf


# Set TensorFlow to use GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU device 0

# Train the model with GPU
history3 = model3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=25, verbose=1,
                       callbacks=callbacks_list, validation_data=val_generator,
                       validation_steps=validation_steps, class_weight=None,
                       initial_epoch=0, batch_size=batch_size)

In [ ]:
plt.figure(figsize=(20,6))

ax1 = plt.subplot(121)
ax1 = plt.plot(history3.history['loss'])
ax1 = plt.plot(history3.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')

ax2 = plt.subplot(122)
ax2 = plt.plot(history3.history['categorical_accuracy'])
ax2 = plt.plot(history3.history['val_categorical_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower left')

In [ ]:
# Model #4 with GRU

In [ ]:
model4 = Sequential()

#Adding another TimeDistributed MaxPooling2D layer (32)
model4.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=input_shape))
model4.add(TimeDistributed(MaxPooling2D((2,2))))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))

#Adding another TimeDistributed MaxPooling2D layer (64)
model4.add(TimeDistributed(Conv2D(64, (3,3), activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2,2))))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))

#Adding another TimeDistributed GlobalAveragePooling2D layer (64)
model4.add(TimeDistributed(GlobalAveragePooling2D()))
model4.add(TimeDistributed(Dense(64, activation='relu')))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))

##Adding GRU Layer (128) with BatchNormalization
model4.add(GRU(128))
model4.add(BatchNormalization())
model4.add(Dense(5, activation='softmax'))

In [ ]:
print ("Model summary : Test #4 ",model4.summary())

In [ ]:
optimiser = optimizers.Adam(learning_rate=0.01)
model4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
train_generator = generator(train_path, train_doc, batch_size,img_idx_sel, xdim, ydim)
val_generator = generator(val_path, val_doc, batch_size,img_idx_sel, xdim, ydim)
##can be skipped if ran fully
#####
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_name = 'model_init4_' + datetime.now().strftime('%Y-%m-%d_%H_%M_%S') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model4-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]


In [ ]:
import os
import tensorflow as tf


# Set TensorFlow to use GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU device 0

# Train the model with GPU
history4 = model4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1,
                       callbacks=callbacks_list, validation_data=val_generator,
                       validation_steps=validation_steps, class_weight=None,
                       initial_epoch=0, batch_size=batch_size)

In [ ]:
plt.figure(figsize=(24,6))

ax1 = plt.subplot(121)
ax1 = plt.plot(history4.history['loss'])
ax1 = plt.plot(history4.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')

ax2 = plt.subplot(122)
ax2 = plt.plot(history4.history['categorical_accuracy'])
ax2 = plt.plot(history4.history['val_categorical_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower left')

In [ ]:
print(history4.history.keys())
print(history4.history['loss'][-1])
print(history4.history['categorical_accuracy'][-1])
print(history4.history['val_categorical_accuracy'][-1])
print(history4.history['val_loss'][-1])
print(history4.history['learning_rate'][-1])



In [ ]:
# Model #5 with LTSM

In [ ]:
model5 = Sequential()

model5.add(TimeDistributed(Conv2D(8, (3,3), activation='relu'), input_shape=input_shape))

model5.add(BatchNormalization())
model5.add(TimeDistributed(Conv2D(16, (3,3), activation='relu')))

model5.add(BatchNormalization())
model5.add(ConvLSTM2D(8, kernel_size = 3, return_sequences=False))

#Adding BatchNormalization & TimeDistributed relu actived Dense Layer (64)
model5.add(BatchNormalization())
model5.add(TimeDistributed(Dense(64, activation='relu')))

#Adding BatchNormalization & GlobalAveragePooling2D
model5.add(BatchNormalization())
model5.add(GlobalAveragePooling2D())

#Adding Dense Layer - relu (64)
model5.add(Dense(64, activation='relu'))

#Adding Dense Layer - softmax (5)
model5.add(Dense(5, activation='softmax'))

In [ ]:
print ("Model summary : Test #5 ",model5.summary())

In [ ]:
optimiser = optimizers.Adam(learning_rate=0.01)
model5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
train_generator = generator(train_path, train_doc, batch_size,img_idx_sel, xdim, ydim)
val_generator = generator(val_path, val_doc, batch_size,img_idx_sel, xdim, ydim)
##can be skipped if ran fully
#####
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_name = 'model_init5_' + datetime.now().strftime('%Y-%m-%d_%H_%M_%S') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model5-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]


In [ ]:
# Set TensorFlow to use GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU device 0

# Train the model with GPU
history5 = model5.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1,
                       callbacks=callbacks_list, validation_data=val_generator,
                       validation_steps=validation_steps, class_weight=None,
                       initial_epoch=0, batch_size=batch_size)

In [ ]:
plt.figure(figsize=(24,6))

ax1 = plt.subplot(121)
ax1 = plt.plot(history5.history['loss'])
ax1 = plt.plot(history5.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')

ax2 = plt.subplot(122)
ax2 = plt.plot(history5.history['categorical_accuracy'])
ax2 = plt.plot(history5.history['val_categorical_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower left')

In [2]:
#### The best model is Model #4 for the below 2 reasons :
    ### i) The difference between Training & Validation loss is the least
    ### ii) Model accuracy is the best out of all other models

In [ ]:
###Re-training model #4 / best model with tunned parameters to see if there is any performance improvement or not.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, TimeDistributed, GlobalAveragePooling2D, Dense, GRU

best_model = Sequential()

# Adding a TimeDistributed Convolutional layer with 64filters
best_model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu'), input_shape=input_shape))
# Adding a TimeDistributed MaxPooling2D layer
best_model.add(TimeDistributed(MaxPooling2D((2, 2))))
best_model.add(BatchNormalization())
best_model.add(Dropout(0.25))

# Adding another TimeDistributed Convolutional layer with 128filters
best_model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
# Adding another TimeDistributed MaxPooling2D layer
best_model.add(TimeDistributed(MaxPooling2D((2, 2))))
best_model.add(BatchNormalization())
best_model.add(Dropout(0.25))


# Adding a TimeDistributed GlobalAveragePooling2D
best_model.add(TimeDistributed(GlobalAveragePooling2D()))
best_model.add(TimeDistributed(Dense(128, activation='relu')))
best_model.add(BatchNormalization())
best_model.add(Dropout(0.25))



# Adding a GRU layer with 256 units
best_model.add(GRU(256))
best_model.add(BatchNormalization())
best_model.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.Adam(learning_rate=0.01)
best_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
train_generator = generator(train_path, train_doc, batch_size,img_idx_sel, xdim, ydim)
val_generator = generator(val_path, val_doc, batch_size,img_idx_sel, xdim, ydim)
##can be skipped if ran fully
#####
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_name = 'best_model' + datetime.now().strftime('%Y-%m-%d_%H_%M_%S') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'best_model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]


In [ ]:
# Set TensorFlow to use GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU device 0

# Train the model with GPU
best_model_history = best_model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=35, verbose=1,
                       callbacks=callbacks_list, validation_data=val_generator,
                       validation_steps=validation_steps, class_weight=None,
                       initial_epoch=0, batch_size=batch_size)

In [ ]:
plt.figure(figsize=(24,6))

ax1 = plt.subplot(121)
ax1 = plt.plot(best_model_history.history['loss'])
ax1 = plt.plot(best_model_history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower left')

ax2 = plt.subplot(122)
ax2 = plt.plot(best_model_history.history['categorical_accuracy'])
ax2 = plt.plot(best_model_history.history['val_categorical_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower left')

In [ ]:
### The tweaked model is having lower accuracy than model4

In [ ]:
print(history4.history.keys())
print(history4.history['loss'][-1])
print(history4.history['categorical_accuracy'][-1])
print(history4.history['val_categorical_accuracy'][-1])
print(history4.history['val_loss'][-1])
print(history4.history['learning_rate'][-1])



In [ ]:
print(best_model_history.history.keys())
print(best_model_history.history['loss'][-1])
print(best_model_history.history['categorical_accuracy'][-1])
print(best_model_history.history['val_categorical_accuracy'][-1])
print(best_model_history.history['val_loss'][-1])
print(best_model_history.history['learning_rate'][-1])

### The best model is Model #4 for the below 2 reasons :
  #### i)  The difference between Training & Validation loss is the least.
  #### ii) Model accuracy is the best out of all other models.

## Please use model4 for test result validation as the tweaked model(best_model) is having marginally lesser accuracy than model4

# Thank you 